## Benchmark experiment on Deep Neural Network based PRSs
This notebook collects all the code needed to run the benchmark experiment on the algorithm named DNN-Badre in *Massi, Franco et al., Learning High-Order Interactions for Polygenic Risk Prediction (2022)*.

The original paper by Badre et al. can be accessed from here: https://www.nature.com/articles/s10038-020-00832-7. The neural network model is implemented according to the specifics thereby detailed.

*Remark* aside from the hiprs package, **this notebook requires the Pytorch library** to be executed.

In [ ]:
# Install the 'hiprs' package directly from the GitHub repository.
# This package is being fetched and installed using the 'pip' command and a GitHub link.
!pip install git+https://github.com/NicolaRFranco/hiprs.git

# Install additional popular data science and machine learning packages using pip.
# - numpy: used for numerical operations and handling arrays.
# - pandas: provides data structures and data analysis tools (often used for dataframes).
# - scipy: library for scientific and technical computing.
# - scikit-learn: popular machine learning library in Python.
# - mlxtend: an extension library for machine learning with additional tools and utilities.
!pip install numpy pandas scipy scikit-learn mlxtend


  Cloning https://github.com/NicolaRFranco/hiprs.git to /tmp/pip-req-build-7gcbbqyf
  Running command git clone --filter=blob:none --quiet https://github.com/NicolaRFranco/hiprs.git /tmp/pip-req-build-7gcbbqyf
  Resolved https://github.com/NicolaRFranco/hiprs.git to commit 5cd7a99b6cb984ac2e18ec9faad28821b6de8b51
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.8 MB/s eta 0:00:00
  Created wheel for hiprs---Nicola-R-Franco: filename=hiprs_Nicola_R_Franco-0.1.2-py3-none-any.whl size=19010 sha256=1a691bb5459a92de3e0207d052c605adbcf9eec51a05a62f4120f9ff36f1198f
  Stored in directory: /tmp/pip-ephem-wheel-cache-znn43h_v/wheels/a5/2c/6b/774b56c914976bd40103e5ad00d41d5e71789017fdadde9217
Successfully built hiprs---Nicola-R-Franco


In [ ]:
import torch # Pytorch library for building DNN models
from hiprs import snps # Auxiliary library for data simulation

In [ ]:
# Data generation
seed = 5 # Refers to the simulated dataset analyzed in the paper for the purpose of model interpretability (cf. Fig. 5)
#In general, seeds 0 to 29 correspond to the 30 datasets analyzed in the paper.

ntrain, ntest = 1000, 500 # Number of observations
p = 15 # Number of SNPs

dataset = snps.generate(n = ntrain + ntest, p = p, noise = 0.01, seed = seed)
dataset

,SNP1,SNP2,SNP3,SNP4,SNP5,SNP6,SNP7,SNP8,SNP9,SNP10,SNP11,SNP12,SNP13,SNP14,SNP15,Outcome
0,1,1,0,0,1,2,2,2,1,1,0,2,1,0,0,0
1,1,1,0,2,2,2,1,1,1,1,0,0,0,0,1,1
2,0,0,0,2,0,2,0,2,2,0,1,2,0,0,0,0
3,1,1,0,2,2,2,1,0,0,0,1,0,2,2,2,1
4,2,2,0,0,2,0,1,0,2,2,0,1,1,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1,1,0,0,2,2,2,0,0,2,1,2,1,2,2,0
1496,2,2,0,0,0,2,0,2,1,0,0,1,2,1,1,0
1497,1,1,0,1,1,0,0,2,2,0,2,1,1,2,0,1
1498,2,2,0,1,1,2,1,2,0,1,2,2,0,0,2,0


**Auxiliary functions and classes for handling DNNs**

In [ ]:
# Importing required modules from the hiprs library.
# Classifier: A base class for classifier models used to structure the DNN class.
# Clock: A utility class for measuring time (though it's not used in this code snippet).
from hiprs.scores import Classifier, Clock

# Defining a new class 'DNN' that inherits (or extends) functionality from the Classifier class.
# This class will be specifically designed for working with deep neural network (DNN) models.
class DNN(Classifier):

    # The __init__ method is a special method that initializes an object of the DNN class.
    # This method runs automatically when a new instance of DNN is created.
    def __init__(self, model):
        # The 'model' parameter is a deep neural network model that gets assigned to 'self.dnn'.
        self.dnn = model
        # 'self.time' is set to 0, to later store the time taken to fit/train the model.
        self.time = 0

    # Defining the 'predict' method to generate predictions on input data 'x'.
    # This method makes predictions without updating any model parameters.
    def predict(self, x):
        # 'torch.no_grad()' prevents gradient calculations, saving memory and speeding up predictions.
        with torch.no_grad():
            # Pass the input 'x' through the DNN model, reshape the output, and move it to CPU.
            # The output is converted to a NumPy array, which is commonly used for data handling.
            return self.dnn(x).view(-1).cpu().numpy()

    # Defining the 'fittingtime' method, which simply returns the value of 'self.time'.
    # This method can be used to get the time taken for the model fitting process.
    def fittingtime(self):
        return self.time


In [ ]:
# Import necessary libraries
from random import shuffle  # Shuffle is used to randomize the order of data for training
import numpy as np  # Import numpy for array and data handling
import torch  # PyTorch library for deep learning and tensor operations
import torch.optim as optim  # Import PyTorch's optimizers for model training
from IPython.display import clear_output  # Used to clear output display in Jupyter notebooks
import time  # Import time library for tracking elapsed time

# Define the cross-entropy loss function
# Cross-entropy loss is commonly used for classification tasks
def lossfunction(ypredicted, ytrue):
    # Return the cross-entropy between predicted and true values with small adjustments to avoid math errors
    return -(ytrue * (ypredicted + 1e-10).log() + (1.0 - ytrue) * (1.0 - ypredicted + 1e-10).log()).sum()

# Define the error function for model evaluation
# L1 error (mean absolute error) calculates the average difference between predictions and actual values
def error(ypredicted, ytrue):
    return (ypredicted - ytrue).abs().mean()

# Training function for the model
# Trains a given model using specified training data, test data, loss function, optimizer, learning rate, and epochs
def train(model, train_data, test_data, lossf, optim, lr, epochs, minibatches=None):
    # Access the deep neural network (DNN) part of the model
    dnn = model.dnn
    ntrain, ntest = len(train_data), len(test_data)  # Number of training and test samples

    # Set the number of samples in each minibatch (default is full batch if not specified)
    if minibatches is None:
        minibatches = ntrain

    # Initialize the optimizer with the model parameters and learning rate
    optimizer = optim(dnn.parameters(), lr=lr)

    # Function to display training progress and errors
    def feedback(epoch, mret, mrev):
        clear_output(wait=True)  # Clears output for cleaner display
        print("%s\nEpoch\tTrain Error\tTest Error")
        print("%d\t%.2e\t%.2e" % (epoch, mret, mrev))

    # Initialize a clock to measure training time
    timer = Clock()
    indexes = list(np.arange(ntrain))  # Create a list of indexes for training samples
    timer.start()  # Start the timer

    # Lists to store training and validation errors for each epoch
    terrors = []
    verrors = []

    # Main training loop that iterates over the number of epochs
    for e in range(epochs):
        shuffle(indexes)  # Shuffle training indexes for each epoch
        # Divide data into minibatches based on specified batch size
        batches = [indexes[(i * minibatches):((i + 1) * minibatches)] for i in range(ntrain // minibatches)]

        # Calculate and store training and test errors without updating model parameters
        with torch.no_grad():
            mret = error(dnn(train_data[:, :-1]), train_data[:, [-1]]).item()  # Training error
            mrev = error(dnn(test_data[:, :-1]), test_data[:, [-1]]).item()  # Test error
            terrors.append(mret)
            verrors.append(mrev)
            feedback(e, mret, mrev)  # Print errors for current epoch

        # Iterate over each minibatch to perform gradient descent
        for minibatch in batches:
            x = train_data[minibatch, :-1]  # Features (input data)
            y = train_data[minibatch, [-1]]  # Labels (target values)

            # Closure function for computing loss and updating model parameters
            def closure():
                optimizer.zero_grad()  # Reset gradients
                loss = lossf(dnn(x), y)  # Compute loss between predictions and actual values
                loss.backward()  # Backpropagate gradients
                return loss

            optimizer.step(closure)  # Perform optimization step to update model weights

    timer.stop()  # Stop the timer after training is complete

    # Final evaluation of training and test errors after training
    with torch.no_grad():
        mret = error(dnn(train_data[:, :-1]), train_data[:, [-1]]).item()  # Final training error
        mrev = error(dnn(test_data[:, :-1]), test_data[:, [-1]]).item()  # Final test error
        terrors.append(mret)
        verrors.append(mrev)
        feedback(e, mret, mrev)

    # Print total elapsed training time
    print("Training complete. Elapsed time: %s." % (timer.elapsedTime()))

    # Store training time in the model for reference
    model.time = timer.elapsed()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from random import shuffle
import numpy as np
import torch
import torch.optim as optim
from IPython.display import clear_output
import time

# Cross-Entropy loss function
def lossfunction(ypredicted, ytrue):
    # Compute cross-entropy loss with a small epsilon for numerical stability
    return -(ytrue * (ypredicted + 1e-10).log() + (1.0 - ytrue) * (1.0 - ypredicted + 1e-10).log()).sum()

# Error function for model evaluation
def error(ypredicted, ytrue):
    # Calculates mean absolute error (L1 error) between predicted and true values
    return (ypredicted - ytrue).abs().mean().item()

# Timer class to measure training time
class Clock:
    def __init__(self):
        self.start_time = None  # Initialize without a start time

    def start(self):
        # Start the timer by noting the current time
        self.start_time = time.time()

    def stop(self):
        # Stop the timer and print elapsed time if it was started
        if self.start_time:
            elapsed = time.time() - self.start_time
            print(f"Elapsed time: {elapsed:.2f} seconds.")
        else:
            print("Timer was not started.")

# Training function to train the model on given dataset
def train(model, train_data, test_data, lossf, optim_class, lr, epochs, minibatches=None):
    dnn = model.dnn  # Access the model's neural network component
    ntrain, ntest = len(train_data), len(test_data)  # Number of training and testing samples

    # Set the number of mini-batches; default is full dataset (batch gradient descent)
    if minibatches is None:
        minibatches = ntrain

    optimizer = optim_class(dnn.parameters(), lr=lr)  # Initialize the optimizer with learning rate

    # Inline function to display training feedback
    def feedback(epoch, mret, mrev):
        clear_output(wait=True)  # Clear previous output for cleaner display
        print(f"Epoch\tTrain Error\tTest Error")
        print(f"{epoch}\t{mret:.2e}\t{mrev:.2e}")  # Print current epoch, train error, and test error

    # Convert data to torch tensors if they aren't already
    if not isinstance(train_data, torch.Tensor):
        train_data = torch.tensor(train_data, dtype=torch.float32)
    if not isinstance(test_data, torch.Tensor):
        test_data = torch.tensor(test_data, dtype=torch.float32)

    # Initialize timer and lists to store errors for analysis
    timer = Clock()
    timer.start()
    terrors = []  # List to track training errors per epoch
    verrors = []  # List to track validation errors per epoch

    # Main training loop for the specified number of epochs
    for e in range(epochs):
        indexes = list(np.arange(ntrain))  # List of training sample indices
        shuffle(indexes)  # Shuffle the training indices for randomness in each epoch
        batches = [indexes[i:i + minibatches] for i in range(0, ntrain, minibatches)]  # Create mini-batches

        # Evaluate and record training and test error at the start of each epoch
        with torch.no_grad():
            mret = error(dnn(train_data[:, :-1]), train_data[:, [-1]])  # Training error
            mrev = error(dnn(test_data[:, :-1]), test_data[:, [-1]])    # Test error
            terrors.append(mret)  # Append current train error
            verrors.append(mrev)  # Append current test error
            feedback(e, mret, mrev)  # Display feedback

        # Iterate over each mini-batch for training
        for minibatch in batches:
            x = train_data[minibatch, :-1]  # Features for current batch
            y = train_data[minibatch, [-1]]  # Labels for current batch

            # Define a closure function for optimizer
            def closure():
                optimizer.zero_grad()  # Reset gradients for each batch
                loss = lossf(dnn(x), y)  # Compute loss for the batch
                loss.backward()  # Backpropagate the error
                return loss

            optimizer.step(closure)  # Update model parameters

    timer.stop()  # Stop timer and print elapsed time

    # Final evaluation after all epochs are complete
    with torch.no_grad():
        mret = error(dnn(train_data[:, :-1]), train_data[:, [-1]])  # Final training error
        mrev = error(dnn(test_data[:, :-1]), test_data[:, [-1]])    # Final test error
        terrors.append(mret)  # Record final train error
        verrors.append(mrev)  # Record final test error
        feedback(e, mret, mrev)  # Display final feedback

    print("Training complete.")  # Indicate end of training
    return terrors, verrors  # Return lists of training and test errors for analysis


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Model fitting and results**

In [ ]:
# Import pandas library for data handling
import pandas as pd

# Splitting dataset into training and testing sets
ntrain, ntest = 1000, 500  # Number of samples for training and testing
tdata, vdata = dataset.iloc[:ntrain, :], dataset.iloc[ntrain:, :]  # Splitting the dataset

# One-hot encoding the SNPs data to make it suitable for neural network processing
train_data = pd.get_dummies(tdata.astype('category')).values  # Encoding training data
test_data = pd.get_dummies(vdata.astype('category')).values  # Encoding test data
mask = [True] * train_data.shape[1]  # Mask for selecting relevant columns
mask[-2] = False  # Exclude the second-to-last column (assumed to be unnecessary)

# Transfer data to GPU for faster computations
train_data = torch.tensor(train_data[:, mask], dtype=torch.float, device=torch.device("cuda:0"))
test_data = torch.tensor(test_data[:, mask], dtype=torch.float, device=torch.device("cuda:0"))

# Defining a Deep Neural Network with Sigmoid activation in the final layer
class Sigmoid(torch.nn.Module):
    def forward(self, x):
        return torch.sigmoid(x)  # Apply sigmoid activation to the output

# Constructing the DNN architecture with several layers and activation functions
dnn = torch.nn.Sequential(
    torch.nn.Linear(train_data.shape[1] - 1, 100),  # First hidden layer
    torch.nn.LeakyReLU(0.2),  # Leaky ReLU activation
    torch.nn.Linear(100, 25),  # Second hidden layer
    torch.nn.LeakyReLU(0.2),  # Leaky ReLU activation
    torch.nn.Linear(25, 5),  # Third hidden layer
    torch.nn.LeakyReLU(0.2),  # Leaky ReLU activation
    torch.nn.Linear(5, 1),  # Output layer
    Sigmoid()  # Sigmoid activation for binary classification
)

# He initialization for layers to improve convergence, particularly with ReLU-based layers
for layer in dnn:
    if isinstance(layer, torch.nn.Linear):
        with torch.no_grad():
            # Kaiming (He) initialization for weights of Linear layers
            torch.nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='leaky_relu', a=0.2)

# Transfer the DNN model to GPU for faster processing
dnn.cuda()

# Instantiate DNN model wrapper for training
badre = DNN(dnn)

# Optimizing the model with Adam optimizer and specified learning rate, mini-batches, and epochs
train(badre, train_data, test_data, lossfunction, torch.optim.Adam, lr=1e-3, minibatches=10, epochs=200)

# Evaluate model performance using AUC and compute total fitting time
auc = badre.auc(test_data[:, :-1], test_data[:, -1].cpu().numpy())  # Calculate AUC for test data
time = badre.fittingtime()  # Retrieve the fitting time

# Output the model's training time and AUC score for the test set
print("Model trained. Elapsed time: %.2f seconds." % time)
print("AUC (test set): %.2f." % auc)


Epoch	Train Error	Test Error
199	2.71e-01	2.76e-01
Training complete.
Model trained. Elapsed time: 0.00 seconds.
AUC (test set): 0.92.
